In [82]:
import numpy as np
import pandas as pd
import seaborn as sns
import string
import pprint

import matplotlib.pyplot as plt
%matplotlib inline

# For lemmatisation
import spacy          
import nltk

# gensim for LDA 
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel




In [83]:
df = pd.read_csv('/content/Product Review Data.csv')
#df.dropna(axis=0, inplace=True)
#df=df.dropna().reset_index(drop=True)
df['reviews.rating'] = df['reviews.rating'].fillna(0)
df.head()

,id,asins,brand,categories,colors,dateAdded,dateUpdated,dimension,ean,keys,...,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,sizes,upc,weight
0,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I initially had trouble deciding between the p...,"Paperwhite voyage, no regrets!",NaN,NaN,Cristina M,NaN,NaN,205 grams
1,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,Allow me to preface this with a little history...,One Simply Could Not Ask For More,NaN,NaN,Ricky,NaN,NaN,205 grams
2,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,4.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I am enjoying it so far. Great for reading. Ha...,Great for those that just want an e-reader,NaN,NaN,Tedd Gardiner,NaN,NaN,205 grams
3,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I bought one of the first Paperwhites and have...,Love / Hate relationship,NaN,NaN,Dougal,NaN,NaN,205 grams
4,AVpe7AsMilAPnD_xQ78G,B00QJDU3KY,Amazon,"Amazon Devices,mazon.co.uk",NaN,2016-03-08T20:21:53Z,2017-07-18T23:52:58Z,169 mm x 117 mm x 9.1 mm,NaN,kindlepaperwhite/b00qjdu3ky,...,5.0,https://www.amazon.com/Kindle-Paperwhite-High-...,I have to say upfront - I don't like coroporat...,I LOVE IT,NaN,NaN,Miljan David Tanic,NaN,NaN,205 grams


In [84]:
df.isnull().values.any()


True

In [85]:
df.isna().any()

id                      False
asins                   False
brand                   False
categories              False
colors                   True
dateAdded               False
dateUpdated             False
dimension                True
ean                      True
keys                    False
manufacturer             True
manufacturerNumber       True
name                    False
prices                  False
reviews.date             True
reviews.doRecommend      True
reviews.numHelpful       True
reviews.rating          False
reviews.sourceURLs      False
reviews.text            False
reviews.title            True
reviews.userCity         True
reviews.userProvince     True
reviews.username         True
sizes                    True
upc                      True
weight                   True
dtype: bool

In [86]:
print(df.shape)

(1597, 27)


In [87]:
# tokenize using gensim simple_preprocess
def sent_to_words(sentences, deacc=True): # deacc=True removes punctuations
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence)))  

# to list
data = df['reviews.text'].values.tolist()
data_words = list(sent_to_words(data))

In [88]:
print(data_words[4])

['have', 'to', 'say', 'upfront', 'don', 'like', 'coroporate', 'hermetically', 'closed', 'stuff', 'like', 'anything', 'by', 'apple', 'or', 'in', 'this', 'case', 'amazon', 'like', 'having', 'devices', 'on', 'which', 'can', 'put', 'anything', 'want', 'and', 'use', 'it', 'but', 'was', 'fairly', 'happy', 'user', 'of', 'nook', 'touch', 'for', 'several', 'years', 'but', 'couldn', 'use', 'all', 'its', 'functionalities', 'since', 'live', 'in', 'serbia', 'then', 'lost', 'the', 'nook', 'and', 'since', 'no', 'other', 'devices', 'can', 'actually', 'be', 'fully', 'used', 'in', 'serbia', 'buying', 'books', 'with', 'them', 'using', 'their', 'online', 'capabilities', 'except', 'the', 'kindle', 'and', 'since', 'no', 'one', 'except', 'amazon', 'ships', 'to', 'serbia', 'and', 'since', 've', 'actually', 'been', 'happy', 'amazon', 'customer', 'since', 'over', 'friends', 'accounts', 'and', 'since', 'through', 'my', 'own', 'and', 'since', 'the', 'kindle', 'definitely', 'has', 'the', 'best', 'technology', 'why

In [89]:
# create list of stop words
# string.punctuation (from the 'string' module) contains a list of punctuations
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english') + list(string.punctuation)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
# Removing stopwords and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [91]:
# call functions

# remove stop words
data_words_nostops = remove_stopwords(data_words)

# initialize spacy 'en' model, use only tagger since we don't need parsing or NER 
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_nostops, allowed_postags=['NOUN', 'ADJ'])


In [92]:
print(data_lemmatized[4])

['coroporate', 'stuff', 'apple', 'case', 'device', 'happy', 'user', 'nook', 'touch', 'several', 'year', 'functionality', 'live', 'serbia', 'nook', 'device', 'serbia', 'book', 'online', 'capability', 'kindle', 'amazon', 'ship', 'happy', 'amazon', 'customer', 'friend', 'kindle', 'good', 'technology', 'buy', 'many', 'review', 'screen', 'light', 'paperwhite', 'similar', 'device', 'problem', 'mine', 'light', 'fine', 'black', 'blotch', 'picture', 'bottom', 'screen', 'black', 'plastic', 'casing', 'picture', 'light', 'blotch', 'light', 'paperwhite', 'screen', 'marvelous', 'fine', 'store', 'day', 'happy', 'guy', 'book', 'amazon', 'book', 'fine', 'thing', 'amazon', 'future', 'upgrade', 'kindle', 'treat', 'book', 'one', 'funcion', 'quote', 'goodread', 'size', 'perfect', 'hand', 'light', 'eye', 'dark', 'light', 'tab', 'packaging', 'fine', 'problem', 'battery', 'life']


In [93]:
# After lemmatization
print(' '.join(data_lemmatized[4]))

coroporate stuff apple case device happy user nook touch several year functionality live serbia nook device serbia book online capability kindle amazon ship happy amazon customer friend kindle good technology buy many review screen light paperwhite similar device problem mine light fine black blotch picture bottom screen black plastic casing picture light blotch light paperwhite screen marvelous fine store day happy guy book amazon book fine thing amazon future upgrade kindle treat book one funcion quote goodread size perfect hand light eye dark light tab packaging fine problem battery life


In [94]:
# create dictionary and corpus

# create dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create corpus
corpus = [id2word.doc2bow(text) for text in data_lemmatized]

In [97]:
# Readable format of corpus (term,frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('adjustment', 1),
  ('auto', 1),
  ('basis', 1),
  ('case', 1),
  ('certain', 1),
  ('change', 1),
  ('custom', 1),
  ('day', 2),
  ('delivery', 1),
  ('dollar', 1),
  ('easy', 1),
  ('expense', 1),
  ('extra', 1),
  ('fine', 1),
  ('friend', 1),
  ('glad', 1),
  ('great', 1),
  ('hard', 1),
  ('international', 1),
  ('jump', 1),
  ('level', 1),
  ('light', 3),
  ('model', 1),
  ('money', 1),
  ('need', 1),
  ('option', 1),
  ('page', 1),
  ('paperwhite', 4),
  ('party', 1),
  ('press', 1),
  ('pricey', 1),
  ('reading', 1),
  ('receptive', 1),
  ('regret', 1),
  ('review', 1),
  ('screen', 1),
  ('sensitive', 1),
  ('service', 1),
  ('setting', 2),
  ('shipping', 2),
  ('specific', 2),
  ('spending', 1),
  ('thing', 1),
  ('third', 1),
  ('time', 3),
  ('touch', 1),
  ('tracking', 1),
  ('trouble', 1),
  ('use', 1),
  ('voyage', 3),
  ('week', 1),
  ('worry', 1)]]

In [98]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=1000,
                                           passes=100,
                                           alpha=0.1,
                                           per_word_topics=True)

In [100]:
# coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.339746043452718


In [99]:
# print the 10 topics
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.030*"tap" + 0.027*"speaker" + 0.025*"good" + 0.024*"great" + 0.020*"dot" '
  '+ 0.019*"sound" + 0.017*"echo" + 0.015*"portable" + 0.013*"cover" + '
  '0.012*"quality"'),
 (1,
  '0.029*"device" + 0.023*"content" + 0.022*"use" + 0.021*"roku" + 0.019*"hdx" '
  '+ 0.015*"prime" + 0.015*"year" + 0.014*"video" + 0.014*"great" + '
  '0.014*"box"'),
 (2,
  '0.027*"kindle" + 0.021*"book" + 0.019*"fire" + 0.016*"amazon" + '
  '0.015*"screen" + 0.012*"paperwhite" + 0.011*"tablet" + 0.011*"hdx" + '
  '0.010*"light" + 0.010*"page"'),
 (3,
  '0.038*"music" + 0.023*"voice" + 0.020*"great" + 0.017*"time" + '
  '0.017*"amazon" + 0.015*"easy" + 0.015*"speaker" + 0.014*"phone" + '
  '0.013*"thing" + 0.012*"comcast"'),
 (4,
  '0.068*"headphone" + 0.030*"apple" + 0.029*"ear" + 0.024*"bud" + '
  '0.023*"people" + 0.020*"sound" + 0.019*"nice" + 0.018*"magnet" + '
  '0.015*"review" + 0.015*"year"'),
 (5,
  '0.035*"fire" + 0.022*"tv" + 0.021*"prime" + 0.018*"app" + 0.015*"amazon" + '
  '0.013*"new" +

In [101]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(binary=True)
vectorizer.fit(df['reviews.text'])

CountVectorizer(binary=True)

In [102]:
len(vectorizer.vocabulary_)

6469

In [114]:
from sklearn.model_selection import train_test_split
#x, x_test, y, y_test = train_test_split(df, df['reviews.rating'], stratify=df['reviews.rating'], test_size=0.25, random_state=42)


df_train, df_test = train_test_split(df, test_size=0.2, shuffle=True)

In [127]:
X_train = vectorizer.transform(df_train['reviews.text'])
X_test = vectorizer.transform(df_test['reviews.text'])

X_train.shape, X_test.shape

((1277, 6469), (320, 6469))

In [126]:
df_train['reviews.rating'].shape

(1277,)

In [116]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB().fit(X_train, df_train['reviews.rating'])

In [117]:
np.exp(clf.class_log_prior_)

array([0.25998434, 0.02584182, 0.02505873, 0.07361002, 0.15113547,
       0.46436962])

In [118]:
from sklearn.metrics import classification_report

predicts = clf.predict(X_train)
print(classification_report(df_train['reviews.rating'], predicts))

              precision    recall  f1-score   support

         0.0       0.61      0.67      0.64       332
         1.0       1.00      0.03      0.06        33
         2.0       0.40      0.12      0.19        32
         3.0       0.71      0.39      0.51        94
         4.0       0.55      0.20      0.30       193
         5.0       0.61      0.81      0.70       593

    accuracy                           0.61      1277
   macro avg       0.65      0.37      0.40      1277
weighted avg       0.61      0.61      0.58      1277



In [128]:
from sklearn.svm import SVC

param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,df_train['reviews.rating'])

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   1.6s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   1.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   1.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   1.5s
[CV] END .........................C=0.1, gamma=1, kernel=rbf; total time=   1.5s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   1.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   1.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   1.3s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   1.2s
[CV] END ........................C=0.1, gamma=1, kernel=poly; total time=   1.2s
[CV] END .....................C=0.1, gamma=1, kernel=sigmoid; total time=   1.2s
[CV] END .....................C=0.1, gamma=1, k

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'poly', 'sigmoid']},
             verbose=2)

In [129]:
from sklearn.metrics import classification_report, confusion_matrix 
grid_predictions = grid.predict(X_test)
print(confusion_matrix(df_test['reviews.rating'],grid_predictions))
print(classification_report(df_test['reviews.rating'],grid_predictions))

[[ 53   0   0   6   1  28]
 [  2   0   0   0   0   7]
 [  0   0   0   0   0   2]
 [  6   0   0  11   0  13]
 [  8   0   0   0   0  35]
 [ 27   0   0   0   0 121]]
              precision    recall  f1-score   support

         0.0       0.55      0.60      0.58        88
         1.0       0.00      0.00      0.00         9
         2.0       0.00      0.00      0.00         2
         3.0       0.65      0.37      0.47        30
         4.0       0.00      0.00      0.00        43
         5.0       0.59      0.82      0.68       148

    accuracy                           0.58       320
   macro avg       0.30      0.30      0.29       320
weighted avg       0.48      0.58      0.52       320



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [108]:
df.describe()

,ean,reviews.numHelpful,reviews.rating,reviews.userCity,reviews.userProvince,sizes,upc
count,8.980000e+02,900.000000,1597.000000,0.0,0.0,0.0,8.980000e+02
mean,8.446269e+11,83.584444,3.212899,NaN,NaN,NaN,8.446269e+11
std,3.391252e+09,197.150238,2.110603,NaN,NaN,NaN,3.391252e+09
min,8.420000e+11,0.000000,0.000000,NaN,NaN,NaN,8.420000e+11
25%,8.420000e+11,0.000000,0.000000,NaN,NaN,NaN,8.420000e+11
50%,8.420000e+11,0.000000,4.000000,NaN,NaN,NaN,8.420000e+11
75%,8.490000e+11,34.000000,5.000000,NaN,NaN,NaN,8.490000e+11
max,8.490000e+11,997.000000,5.000000,NaN,NaN,NaN,8.490000e+11
